In [3]:
%pip install conllu
%pip install stanza

import conllu
import pandas as pd
import stanza
from collections import defaultdict, Counter
from itertools import pairwise
import math

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Czech-PDTC/refs/heads/master/cs_pdtc-ud-train-la.conllu -O cs_pdtc-ud-train-la.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Czech-PDTC/refs/heads/master/cs_pdtc-ud-test.conllu -O cs_pdtc-ud-test.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Czech-PDTC/refs/heads/master/cs_pdtc-ud-dev.conllu -O cs_pdtc-ud-dev.conllu

!wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-GUM/refs/heads/master/en_gum-ud-train.conllu -O en_gum-ud-train.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-GUM/refs/heads/master/en_gum-ud-test.conllu -O en_gum-ud-test.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_English-GUM/refs/heads/master/en_gum-ud-dev.conllu -O en_gum-ud-dev.conllu


--2026-02-09 20:35:16--  https://raw.githubusercontent.com/UniversalDependencies/UD_Czech-PDTC/refs/heads/master/cs_pdtc-ud-train-la.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24175776 (23M) [text/plain]
Saving to: ‘cs_pdtc-ud-train-la.conllu’

cs_pdtc-ud-train-la 100%[===================>]  23,06M  34,4MB/s    in 0,7s    

2026-02-09 20:35:19 (34,4 MB/s) - ‘cs_pdtc-ud-train-la.conllu’ saved [24175776/24175776]

--2026-02-09 20:35:19--  https://raw.githubusercontent.com/UniversalDependencies/UD_Czech-PDTC/refs/heads/master/cs_pdtc-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.13

In [4]:
filenames_test = {
    "cs" : "cs_pdtc-ud-test.conllu",
    "en": "en_gum-ud-test.conllu"
}

filenames_dev = {
    "cs" : "cs_pdtc-ud-dev.conllu",
    "en": "en_gum-ud-dev.conllu"
}

filenames_train = {
    "cs" : "cs_pdtc-ud-train-la.conllu",
    "en": "en_gum-ud-train.conllu"
}

filenames = {
    "test": filenames_test,
    "dev": filenames_dev,
    "train": filenames_train
}

In [5]:
corpus = dict()

for name, d in filenames.items():
    corpus[name] = dict()
    for lang, filename in d.items():
        with open(filename, "r") as f:
            corpus[name][lang] = f.read()

In [6]:
nlp_en = stanza.Pipeline(lang='en', processors='tokenize,pos', tokenize_pretokenized=True)
nlp_cs = stanza.Pipeline(lang='cs', processors='tokenize,pos', tokenize_pretokenized=True)


2026-02-09 20:36:12 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2026-02-09 20:36:14 INFO: Downloaded file to /home/patrik/stanza_resources/resources.json
2026-02-09 20:36:14 INFO: Loading these models for language: en (English):
| Processor | Package         |
-------------------------------
| tokenize  | combined        |
| pos       | combined_charlm |

2026-02-09 20:36:14 INFO: Using device: cpu
2026-02-09 20:36:14 INFO: Loading: tokenize
2026-02-09 20:36:14 INFO: Loading: pos
2026-02-09 20:36:23 INFO: Done loading processors!
2026-02-09 20:36:23 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2026-02-09 20:36:23 INFO: Downloaded file to /home/patrik/stanza_resources/resources.json
2026-02-09

In [7]:
# MANUAL FUNCTIONS - FOR DEBUGGING AND STUFF - NOT USED CURRENTLY

def line_number_check(data, text):
    i=0
    for line in data.split("\n"):
        splitted = line.split("\t")
        if len(splitted) == 0 or splitted[0].startswith("#") or "-" in splitted[0] or "." in splitted[0] or splitted[0] == "\n" or splitted[0] == "":
            continue
        i += 1
    print(f"{text} size : {i}")


def word_tag_count(data, text):
    word_tag_pairs = set()
    i = 0
    for line in data.split("\n"):
        splitted = line.split("\t")
        if len(splitted) == 0 or splitted[0].startswith("#") or "-" in splitted[0] or "." in splitted[0] or splitted[0] == "\n" or splitted[0] == "":
            continue
        word_tag_pairs.add((splitted[1], splitted[4]))
        i+=1

    print(f"{text} size : {i}")

def word_tag_count(data, text):
    word_tag_pairs = set()
    i = 0
    for line in data.split("\n"):
        splitted = line.split("\t")
        if len(splitted) == 0 or splitted[0].startswith("#") or "-" in splitted[0] or "." in splitted[0] or splitted[0] == "\n" or splitted[0] == "":
            continue
        word_tag_pairs.add((splitted[1], splitted[4]))
        i+=1

    print(f"{text} size : {i}")
    # print(f"{text} word-tag pairs : {len(word_tag_pairs)}")

# line_number_check(corpus["train"]["en"], "English train")
# line_number_check(corpus["dev"]["cs"], "Czech dev")
# line_number_check(corpus["test"]["cs"], "Czech test")

# word_tag_count(corpus["test"]["en"], "English test")
# word_tag_count("\n\n".join(corpus["test"]["cs"].split("\n\n")[:1500]), "Czech test")

In [8]:
def tagize(filename, text, max_sentences=999_999_999):
    tokens = []
    tags = []
    sentence_count=0
    tag_count = 0
    with open (filename, "r") as f:
        for sentence in conllu.parse_incr(f):
            sentence_tokens = []
            sentence_tags = []
            if sentence_count >= max_sentences:
                break
            for token in sentence:
                if isinstance(token["id"], (list, tuple)) and ("." in token["id"] or "-" in token["id"]):
                    continue
                # print(token["form"], token["upos"])
                tag_count += 1
                sentence_tokens.append(token["form"])
                sentence_tags.append(token["upos"])
            
            sentence_count += 1
            tokens.append(sentence_tokens)
            tags.append(sentence_tags)
    print(text, "sentence count:", sentence_count, "word/tag pairs (lines):", tag_count)
    return tokens, tags


print("BEFORE TRUNCATION TO 1_500 SENTENCES")
pairs = {type: {lang: tagize(filename, f"{type}-{lang}") for lang, filename in d.items()} for type, d in filenames.items()}

BEFORE TRUNCATION TO 1_500 SENTENCES
test-cs sentence count: 20187 word/tag pairs (lines): 305808
test-en sentence count: 1464 word/tag pairs (lines): 28397
dev-cs sentence count: 22666 word/tag pairs (lines): 384431
dev-en sentence count: 1575 word/tag pairs (lines): 28119
train-cs sentence count: 12519 word/tag pairs (lines): 218409
train-en sentence count: 10224 word/tag pairs (lines): 177410


In [9]:
print("TRUNCATING TEST SETS")
pairs["test"] = {lang: tagize(filename, f"test-{lang}", max_sentences=1500) for lang, filename in filenames["test"].items()}
tokens = {type: {lang: pair[0] for lang, pair in d.items()} for type, d in pairs.items()}
tags = {type: {lang: pair[1] for lang, pair in d.items()} for type, d in pairs.items()}

TRUNCATING TEST SETS
test-cs sentence count: 1500 word/tag pairs (lines): 22844
test-en sentence count: 1464 word/tag pairs (lines): 28397


In [10]:
doc_en = nlp_en(tokens["test"]["en"])
doc_cs = nlp_cs(tokens["test"]["cs"][:1500])

In [11]:
def stanza_eval(doc, tokens_sents, gold_sents):
    """tokens_sents: List[List[str]], gold_sents: List[List[str]] (UPOS)"""
    pred_sents = [[w.upos for w in s.words] for s in doc.sentences]

    # accuracy
    correct = total = 0
    for g, p in zip(gold_sents, pred_sents):
        if len(g) != len(p):
            raise ValueError(f"Length mismatch in a sentence: gold={len(g)} pred={len(p)} "
                             "(check MWT/ellipsis filtering + tokenize_pretokenized=True)")
        for gt, pt in zip(g, p):
            total += 1
            correct += (gt == pt)
    acc = 100.0 * correct / total

    # confusion matrix (rows=gold, cols=pred)
    gflat = [t for sent in gold_sents for t in sent]
    pflat = [t for sent in pred_sents for t in sent]
    cm = pd.crosstab(pd.Series(gflat, name="gold"), pd.Series(pflat, name="pred"))
    return acc, cm

en_acc, en_cm = stanza_eval(doc_en, tokens["test"]["en"], tags["test"]["en"])
print(f"Stanza EN test accuracy: {en_acc:.4f}%")
print("Stanza EN confusion matrix below")
display(en_cm)

cs_acc, cs_cm = stanza_eval(doc_cs, tokens["test"]["cs"][:1500], tags["test"]["cs"][:1500])
print(f"Stanza CS test accuracy: {cs_acc:.4f}%")
print("Stanza CS confusion matrix below")
display(cs_cm)


Stanza EN test accuracy: 97.8308%
Stanza EN confusion matrix below


pred,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X
gold,,,,,,,,,,,,,,,,,
ADJ,1755,2,11,0,0,0,0,34,0,0,0,13,0,0,0,18,0
ADP,0,2833,12,0,2,0,0,0,0,1,0,2,0,15,0,0,0
ADV,37,36,1226,0,0,14,6,8,0,0,3,1,0,4,0,1,0
AUX,0,0,0,1520,0,0,0,0,0,0,0,0,0,0,0,13,0
CCONJ,0,0,0,0,987,0,0,0,0,0,0,0,0,1,0,0,0
DET,0,0,1,0,3,2515,1,0,0,0,1,0,0,0,0,0,0
INTJ,1,4,12,0,1,0,155,2,0,0,0,1,0,3,0,2,3
NOUN,25,0,0,1,0,0,2,4806,3,0,1,59,0,0,0,8,1
NUM,0,1,0,0,0,0,0,0,478,0,0,0,0,0,0,0,0


Stanza CS test accuracy: 97.2903%
Stanza CS confusion matrix below


pred,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X
gold,,,,,,,,,,,,,,,,,
ADJ,2572,0,1,0,0,0,0,16,1,0,0,11,0,0,0,0,1
ADP,0,2109,0,0,0,0,0,3,0,0,1,0,0,0,0,1,0
ADV,1,2,878,0,16,1,0,10,0,5,3,0,0,0,0,1,0
AUX,0,0,0,774,0,0,0,0,0,0,0,0,0,0,0,2,0
CCONJ,0,0,12,0,731,0,0,0,0,2,0,0,0,11,0,0,0
DET,1,0,1,0,0,916,0,0,0,0,0,0,0,0,0,1,0
INTJ,0,0,0,0,0,0,14,2,0,3,0,1,0,0,0,0,0
NOUN,12,3,7,0,4,0,4,5525,0,0,0,32,0,0,6,3,4
NUM,0,0,0,0,0,0,0,0,616,0,0,0,0,0,0,0,0


In [13]:
print("TRUNKATING CS DEV SET")
tokens["dev"]["cs"], tags["dev"]["cs"] = tagize(filenames["dev"]["cs"], "dev-cs", max_sentences=1500)

TRUNKATING CS DEV SET
dev-cs sentence count: 1500 word/tag pairs (lines): 21752


In [14]:
# Check sizes
print("Sizes check")
for type, d in tokens.items():
    for lang, sents in d.items():
        print(f"{type}-{lang} tokenized sentences: {len(sents)} tokens : {sum(len(s) for s in sents)}")

Sizes check
test-cs tokenized sentences: 1500 tokens : 22844
test-en tokenized sentences: 1464 tokens : 28397
dev-cs tokenized sentences: 1500 tokens : 21752
dev-en tokenized sentences: 1575 tokens : 28119
train-cs tokenized sentences: 12519 tokens : 218409
train-en tokenized sentences: 10224 tokens : 177410


# Transition model smoothing

In [15]:
BOS = "<s>"
EOS = "</s>"

# UPDATED BIGRAMS SO THERE IS ALSO start_symbol followed by start_symbol (and also end followed by end)
get_unigrams = lambda tokens : Counter(tokens + [EOS])
get_bigrams = lambda tokens : Counter(zip(2 * [BOS] + tokens + [EOS], [BOS] + tokens + 2 * [EOS]))
get_trigrams = lambda tokens : Counter(zip(2 * [BOS] + tokens, [BOS] + tokens + [EOS], tokens + 2 * [EOS]))

def get_all_ngrams(tokens_or_tags, n=1):
    ngrams = {
        1: get_unigrams,
        2: get_bigrams,
        3: get_trigrams
    }[n]
    all = Counter()    
    for sentence in tokens_or_tags:
        all.update(ngrams(sentence))
    return all


In [16]:
# COPIED FROM THE FIRST ASSIGNMENT
# NOT IDEA IF THIS IS CORRECT SINCE I REVIECED NO FEEDBACK :)
# BUT I GOT 92 POINTS SO HOPEFULLY THIS IS OK

def bigram_probab(bigrams, unigrams, wi, wi_1):
    c2 = bigrams[(wi_1, wi)]#  if (wi_1, wi) in bigrams else 0
    c1 = unigrams[wi_1]# if wi_1 in unigrams else 0
    if c1 == 0:
        return 1 / len(unigrams)
    return c2 / c1

def trigram_probab(trigrams, bigrams, unigrams, wi, wi_1, wi_2):
    c3 = trigrams[(wi_2, wi_1, wi)]# if (wi_2, wi_1, wi) in trigrams else 0
    c2 = bigrams[(wi_2, wi_1)]# if (wi_2, wi_1) in bigrams else 0
    if c2 == 0:
        return 1 / len(unigrams)
    return c3 / c2

def smoothed_probab_trigram(trigrams, bigrams, unigrams, wi, wi_1, wi_2, lambdas, N_uni = None):
    N_uni = sum(unigrams.values()) if N_uni == None else N_uni
    lambda0, lambda1, lambda2, lambda3 = lambdas
    p0 = 1 / len(unigrams)
    p1 = unigrams[wi] / N_uni# if wi in unigrams else 1 / sum(unigrams.values())
    p2 = bigram_probab(bigrams, unigrams, wi, wi_1)
    p3 = trigram_probab(trigrams, bigrams, unigrams, wi, wi_1, wi_2)
    return lambda0 * p0 + lambda1 * p1 + lambda2 * p2 + lambda3 * p3


def EM_smoothing(train, heldout, max_iter=100, e = 1e-4):
    lambdas = [0.25, 0.25, 0.25, 0.25]

    # --- ONE MINOR CHANGE FROM THE FIRST ASSIGNMENT ---
    # trigrams_heldout = get_trigrams(heldout)
    # bigrams_heldout = get_bigrams(heldout)
    # unigrams_heldout = get_unigrams(heldout)

    # trigrams_train = get_trigrams(train)
    # bigrams_train = get_bigrams(train)
    # unigrams_train = get_unigrams(train)

    trigrams_heldout = get_all_ngrams(heldout, n=3)
    bigrams_heldout = get_all_ngrams(heldout, n=2)
    unigrams_heldout = get_all_ngrams(heldout, n=1)

    trigrams_train = get_all_ngrams(train, n=3)
    bigrams_train = get_all_ngrams(train, n=2)
    unigrams_train = get_all_ngrams(train, n=1)
    # --- --- ----- ------ ---- --- ----- ---------- ---

    N_uni = sum(unigrams_train.values()) # saves a lot of time lol

    iter = 0
    while iter < max_iter:
        counts = [0.0, 0.0, 0.0, 0.0]

        for (wi_2, wi_1, wi), c in trigrams_heldout.items():
            p0 = 1 / len(unigrams_train)
            p1 = unigrams_train[wi] / N_uni# if wi in unigrams_train else 1 / N_uni
            p2 = bigram_probab(bigrams_train, unigrams_train, wi, wi_1)
            p3 = trigram_probab(trigrams_train, bigrams_train, unigrams_train, wi, wi_1, wi_2)

            p_total = lambdas[0] * p0 + lambdas[1] * p1 + lambdas[2] * p2 + lambdas[3] * p3

            counts[0] += c * (lambdas[0] * p0) / p_total
            counts[1] += c * (lambdas[1] * p1) / p_total
            counts[2] += c * (lambdas[2] * p2) / p_total
            counts[3] += c * (lambdas[3] * p3) / p_total

        new_lambdas = [count / sum(counts) for count in counts]

        # print("Iteration ", iter, " lambdas: ", new_lambdas)

        if all(abs(new_lambdas[i] - lambdas[i]) < e for i in range(4)):
            break

        lambdas = new_lambdas
        iter += 1
    return lambdas

In [17]:
ngrams_train_tag = {type: {i: {lang: get_all_ngrams(tags["train"][lang], n=i) for lang in tags["train"]} for i in range(1, 4)} for type in tags["train"]}

In [18]:
em_lambdas = {lang: EM_smoothing(tags["train"][lang], tags["dev"][lang]) for lang in tags["train"]}

In [19]:
em_lambdas

{'cs': [0.005635399093115278,
  0.005527962553778706,
  0.10105815981569118,
  0.8877784785374149],
 'en': [0.00012675633014343153,
  0.0033791000786512265,
  0.08104221881633325,
  0.915451924774872]}

# Emission model smoothing

In [25]:
from collections import Counter
import math

UNK = "<UNK>"

def train_lexical_model(tokens_train, tags_train, tokens_dev, tags_dev,
                        unk_threshold=1, add_alpha=0.0, eps=1e-6, max_iter=80):
    """
    Learns interpolated emission model with UNK:

      P(w|t) = λ0 * 1/|V|  +  λ1 * P(w)  +  λ2 * P_MLE(w|t)

    - UNK: words with train freq <= unk_threshold become <UNK> in training,
           unseen words at decode time also become <UNK>.
    - Lambdas (λ0,λ1,λ2) learned on DEV with EM-like expected-counts updates.

    Returns dict with:
      - logp_emit(tag, word): log P(word|tag)
      - lambdas: [λ0, λ1, λ2]
      - vocab: normalized vocab (includes <UNK>)
    """

    # --- TRAIN vocab + UNK mapping ---
    wf = Counter(w for sent in tokens_train for w in sent)

    def norm_train(w):
        return w if wf[w] > unk_threshold else UNK

    emit = Counter()         # (t, w)
    tag_count = Counter()    # t
    word_count = Counter()   # w
    vocab = {UNK}
    total = 0

    for words, tags in zip(tokens_train, tags_train):
        for w, t in zip(words, tags):
            w2 = norm_train(w)
            vocab.add(w2)
            emit[(t, w2)] += 1
            tag_count[t] += 1
            word_count[w2] += 1
            total += 1

    V = len(vocab)

    def p0(w):  # uniform over vocab
        return 1.0 / V

    def p1(w):  # unigram P(w)
        denom = total + add_alpha * V
        return (word_count[w] + add_alpha) / denom if denom else 0.0

    def p2(t, w):  # MLE P(w|t)
        denom = tag_count[t] + add_alpha * V
        return (emit[(t, w)] + add_alpha) / denom if denom else 0.0

    def norm_dev(w):
        # unseen -> UNK
        return w if w in vocab else UNK

    # --- learn lambdas on DEV ---
    lambdas = [1/3, 1/3, 1/3]
    for _ in range(max_iter):
        exp = [0.0, 0.0, 0.0]

        for words, tags in zip(tokens_dev, tags_dev):
            for w_raw, t in zip(words, tags):
                w = norm_dev(w_raw)
                probs = [p0(w), p1(w), p2(t, w)]
                mix = [lambdas[i] * probs[i] for i in range(3)]
                Z = sum(mix)
                if Z <= 0:
                    continue
                exp[0] += mix[0] / Z
                exp[1] += mix[1] / Z
                exp[2] += mix[2] / Z

        s = sum(exp)
        new_lambdas = [e / s for e in exp] if s else lambdas
        if max(abs(new_lambdas[i] - lambdas[i]) for i in range(3)) < eps:
            lambdas = new_lambdas
            break
        lambdas = new_lambdas

    def logp_emit(t, w_raw):
        w = w_raw if w_raw in vocab else UNK
        p = lambdas[0] * p0(w) + lambdas[1] * p1(w) + lambdas[2] * p2(t, w)
        return -math.inf if p <= 0.0 else math.log(p)

    return {"vocab": vocab, "lambdas": lambdas, "logp_emit": logp_emit,
            "emit": emit, "tag_count": tag_count, "word_count": word_count}


In [27]:
lex = {}
for lang in ["en", "cs"]:
    lex[lang] = train_lexical_model(
        tokens["train"][lang], tags["train"][lang],
        tokens["dev"][lang],   tags["dev"][lang],
        unk_threshold=1, add_alpha=0.0
    )


# HMM/Viterbi

In [28]:
def viterbi_trigram(words, tagset, logp_trans, logp_emit, BOS="<s>", EOS="</s>"):
    """
    Trigram Viterbi for tags t1..tn given words w1..wn.

    logp_trans(u,v,t) = log P(t | u,v)
    logp_emit(t,w)    = log P(w | t)

    Returns: list of predicted tags length n
    """
    n = len(words)
    if n == 0:
        return []

    tagset = [t for t in tagset if t not in (BOS, EOS)]

    # dp_i[(u,v)] = best score for sequence ending with ... u,v at position i (i>=0)
    dp = {(BOS, BOS): 0.0}
    bp = []  # bp[i][(u,v)] = best prevprev tag (the tag before u)

    for i, w in enumerate(words, start=1):
        new_dp = {}
        new_bp = {}

        for (pp, p), score in dp.items():
            if score == -math.inf:
                continue
            for c in tagset:
                s = score + logp_trans(pp, p, c) + logp_emit(c, w)
                key = (p, c)
                if s > new_dp.get(key, -math.inf):
                    new_dp[key] = s
                    new_bp[key] = pp  # store best prevprev for this (p,c)

        dp = new_dp
        bp.append(new_bp)

    # terminate with two EOS transitions
    best_key = None
    best_score = -math.inf
    for (u, v), score in dp.items():
        s_end = score + logp_trans(u, v, EOS) + logp_trans(v, EOS, EOS)
        if s_end > best_score:
            best_score = s_end
            best_key = (u, v)

    # backtrack
    tags_out = [None] * n
    u, v = best_key
    tags_out[n-2] = u if n >= 2 else v
    tags_out[n-1] = v

    # i indexes 1..n, bp index is i-1
    # For i=n: we know (t_{n-1}, t_n) = (u,v) and bp[n-1][(u,v)] gives t_{n-2}
    for i in range(n, 2, -1):
        pp = bp[i-1][(tags_out[i-2], tags_out[i-1])]
        tags_out[i-3] = pp

    # For n=1, tags_out is [v] already correct; for n=2, loop doesn’t run; ok.
    if n == 1:
        tags_out = [v]

    return tags_out


In [ ]:
def take_first_n_pairs(tokens_sents, tags_sents, n_pairs):
    out_tok, out_tag = [], []
    seen = 0
    for words, tags in zip(tokens_sents, tags_sents):
        if seen >= n_pairs:
            break
        need = n_pairs - seen
        if len(words) <= need:
            out_tok.append(words)
            out_tag.append(tags)
            seen += len(words)
        else:
            out_tok.append(words[:need])
            out_tag.append(tags[:need])
            seen += need
            break
    return out_tok, out_tag

def build_logp_trans_from_train_tags(lang, tags_train_sents):
    uni = get_all_ngrams(tags_train_sents, n=1)
    bi  = get_all_ngrams(tags_train_sents, n=2)
    tri = get_all_ngrams(tags_train_sents, n=3)
    N_uni = sum(uni.values())
    lambdas = em_lambdas[lang]

    def logp_trans(u, v, t):
        p = smoothed_probab_trigram(tri, bi, uni, wi=t, wi_1=v, wi_2=u, lambdas=lambdas, N_uni=N_uni)
        return -math.inf if p <= 0.0 else math.log(p)

    tagset = [t for t in uni.keys() if t not in (BOS, EOS)]
    return logp_trans, tagset

import pandas as pd

def eval_tagging(gold_sents, pred_sents):
    gold_flat, pred_flat = [], []
    for i, (g, p) in enumerate(zip(gold_sents, pred_sents)):
        if len(g) != len(p):
            raise ValueError(f"Sentence {i} length mismatch: gold={len(g)} pred={len(p)}")
        gold_flat.extend(g)
        pred_flat.extend(p)
    acc = 100.0 * sum(gt == pt for gt, pt in zip(gold_flat, pred_flat)) / len(gold_flat)
    cm = pd.crosstab(pd.Series(gold_flat, name="gold"), pd.Series(pred_flat, name="pred"))
    return acc, cm



In [170]:
def take_first_n_tokens(tokens_sents, tags_sents, n_tokens):
    """
    Keeps sentence structure but truncates after n_tokens total tokens.
    """
    out_tokens, out_tags = [], []
    count = 0

    for words, tags in zip(tokens_sents, tags_sents):
        if count >= n_tokens:
            break

        if count + len(words) <= n_tokens:
            out_tokens.append(words)
            out_tags.append(tags)
            count += len(words)
        else:
            # partial sentence
            k = n_tokens - count
            out_tokens.append(words[:k])
            out_tags.append(tags[:k])
            count = n_tokens
            break

    return out_tokens, out_tags

tokens_train_10k_en, tags_train_10k_en = take_first_n_tokens(
    tokens["train"]["en"],
    tags["train"]["en"],
    10_000
)

tokens_train_10k_cs, tags_train_10k_cs = take_first_n_tokens(
    tokens["train"]["cs"],
    tags["train"]["cs"],
    10_000
)


print("Train tokens used:", sum(len(s) for s in tokens_train_10k_en))
print("Train tokens used:", sum(len(s) for s in tokens_train_10k_cs))


Train tokens used: 10000
Train tokens used: 10000


In [171]:
# Tag n-grams from reduced training set
uni_10k_en = get_all_ngrams(tags_train_10k_en, n=1)
bi_10k_en  = get_all_ngrams(tags_train_10k_en, n=2)
tri_10k_en = get_all_ngrams(tags_train_10k_en, n=3)

uni_10k_cs = get_all_ngrams(tags_train_10k_cs, n=1)
bi_10k_cs  = get_all_ngrams(tags_train_10k_cs, n=2)
tri_10k_cs = get_all_ngrams(tags_train_10k_cs, n=3)

# Lambdas still estimated on FULL dev set (important!)
lambdas_10k_en = em_lambdas["en"]
lambdas_10k_cs = em_lambdas["cs"]

N_uni_10k_en = sum(uni_10k_en.values())
N_uni_10k_cs = sum(uni_10k_cs.values())

lex_10k_en = train_lexical_model(
    tokens_train_10k_en,
    tags_train_10k_en,
    unk_threshold=1,
    add_alpha=0.0
)

lex_10k_cs = train_lexical_model(
    tokens_train_10k_cs,
    tags_train_10k_cs,
    unk_threshold=1,
    add_alpha=0.0
)


In [172]:
def logp_trans_10k_en(u, v, t):
    p = smoothed_probab_trigram(
        tri_10k_en, bi_10k_en, uni_10k_en,
        wi=t, wi_1=v, wi_2=u,
        lambdas=lambdas_10k_en,
        N_uni=N_uni_10k_en
    )
    return -math.inf if p <= 0 else math.log(p)

def logp_trans_10k_cs(u, v, t):
    p = smoothed_probab_trigram(
        tri_10k_cs, bi_10k_cs, uni_10k_cs,
        wi=t, wi_1=v, wi_2=u,
        lambdas=lambdas_10k_cs,
        N_uni=N_uni_10k_cs
    )
    return -math.inf if p <= 0 else math.log(p)



tagset_10k_en = [t for t in uni_10k_en if t not in (BOS, EOS)]
tagset_10k_cs = [t for t in uni_10k_cs if t not in (BOS, EOS)]


In [176]:
pred_sents_10k_en = [
    viterbi_trigram(sent, tagset_10k_en, logp_trans_10k_en, lex_10k_en["logp_emit"])
    for sent in tokens["test"]["en"]
]

gold_flat, pred_flat = [], []
for g, p in zip(tags["test"]["en"], pred_sents_10k_en):
    if len(g) != len(p):
        raise ValueError("Length mismatch during evaluation")
    gold_flat.extend(g)
    pred_flat.extend(p)

acc_10k_en = 100.0 * sum(gt == pt for gt, pt in zip(gold_flat, pred_flat)) / len(gold_flat)
print(f"Unsupervised HMM (init, 10k supervised) EN accuracy: {acc_10k_en:.4f}")

print("Unsupervised HMM (init, 10k supervised) EN confusion matrix below")
cm_10k_en = pd.crosstab(
    pd.Series(gold_flat, name="gold"),
    pd.Series(pred_flat, name="pred")
)

display(cm_10k_en)

Unsupervised HMM (init, 10k supervised) EN accuracy: 72.4337
Unsupervised HMM (init, 10k supervised) EN confusion matrix below


pred,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,PRON,PROPN,PUNCT,SCONJ,SYM,VERB,X
gold,,,,,,,,,,,,,,,,,
ADJ,1094,4,52,0,0,0,0,231,7,0,1,213,0,0,2,220,9
ADP,16,2581,24,0,2,2,0,24,0,68,0,32,0,48,0,62,6
ADV,95,64,688,0,1,19,0,177,7,0,36,95,0,16,2,132,4
AUX,21,0,8,1174,0,0,0,39,2,79,3,73,0,0,0,132,2
CCONJ,10,0,39,0,907,1,0,4,0,0,5,8,0,1,1,10,2
DET,15,0,3,0,10,2400,0,11,0,0,38,23,0,12,0,5,4
INTJ,21,8,78,0,0,0,2,21,3,0,1,32,0,2,1,10,5
NOUN,391,0,54,2,0,1,0,3534,27,0,5,565,0,1,4,303,19
NUM,37,0,4,0,0,0,0,76,239,1,22,77,0,0,0,20,3


In [177]:
pred_sents_10k_cs = [
    viterbi_trigram(sent, tagset_10k_cs, logp_trans_10k_cs, lex_10k_cs["logp_emit"])
    for sent in tokens["test"]["cs"]
]

gold_flat, pred_flat = [], []
for g, p in zip(tags["test"]["cs"], pred_sents_10k_cs):
    if len(g) != len(p):
        raise ValueError("Length mismatch during evaluation")
    gold_flat.extend(g)
    pred_flat.extend(p)

acc_10k_cs = 100.0 * sum(gt == pt for gt, pt in zip(gold_flat, pred_flat)) / len(gold_flat)
print(f"Unsupervised HMM (init, 10k supervised) CS accuracy: {acc_10k_cs:.4f}")

print("Unsupervised HMM (init, 10k supervised) CS confusion matrix below")
cm_10k_cs = pd.crosstab(
    pd.Series(gold_flat, name="gold"),
    pd.Series(pred_flat, name="pred")
)

Unsupervised HMM (init, 10k supervised) CS accuracy: 68.5169
Unsupervised HMM (init, 10k supervised) CS confusion matrix below
